In [193]:
import sys
sys.path.append('..')

from sqlalchemy import create_engine
import geopandas as gpd
import numpy as np
import pandas as pd

from shapely.geometry import Polygon
from shapely import wkb

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

In [194]:
datasets = {
    'canha_quemada' : {
        'project' : 65454,
        'dataset' : 51666
    },
    'plan_renovacion' : {
        'project' : 36292,
        'dataset' : 382781
    },
}
datasets

{'canha_quemada': {'project': 65454, 'dataset': 51666},
 'plan_renovacion': {'project': 36292, 'dataset': 382781}}

In [195]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )
    
def get_catastro():
    engine = obtener_engine()
    try:
        query = """
            SELECT idd, geom, unidad_01, unidad_02, 
                unidad_03, unidad_04, unidad_05, 
                variedad, soca, textura, cultivo, area 
            FROM catastro_iag.catastro order by idd
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error get catastro: {e}")
        return gpd.GeoDataFrame()
    return None

def get_lote_by_idd(idd):
    engine = obtener_engine()
    try:
        query = f"""
            SELECT idd, geom, unidad_01, unidad_02, 
                unidad_03, unidad_04, unidad_05, 
                variedad, soca, textura, cultivo, area 
            FROM catastro_iag.catastro where idd = {idd}
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error al obtener lote by id: {e}")
        return gpd.GeoDataFrame()
    return None

def get_catastro_sync():
    engine = obtener_engine()
    try:
        query = """
            SELECT idd, geom, unidad_01, unidad_02, 
                unidad_03, unidad_04, unidad_05, 
                variedad, soca, textura, cultivo, area 
            FROM amigcloud_sync.catastro_amigocloud_sync order by idd
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error get catastro: {e}")
        return gpd.GeoDataFrame()
    return None

def get_registro_sync():
    engine = obtener_engine()
    try:
        query = """
            SELECT * FROM amigcloud_sync.registro_sync WHERE procesado = false limit 1
        """
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"❌ Error al obtener resgistros para sincronizar: {e}")
        return gpd.GeoDataFrame()
    return None

def convertir_a_multipolygon(geometry):
    if isinstance(geometry, Polygon):
        return MultiPolygon([geometry])
    return geometry

def convertir_a_wkb(polygon):
    wkb_data = wkb.dumps(polygon, hex=True)
    return wkb_data

In [196]:
gdf_catastro = get_catastro()
gdf_catastro_sync = get_catastro_sync()

In [197]:
gdf_catastro["idd"] = gdf_catastro["idd"].astype(str)
gdf_catastro["unidad_01"] = gdf_catastro["unidad_01"].astype(str)
gdf_catastro["unidad_02"] = gdf_catastro["unidad_02"].astype(str)
gdf_catastro["unidad_03"] = gdf_catastro["unidad_03"].astype(str)
gdf_catastro["unidad_04"] = gdf_catastro["unidad_04"].astype(str)
gdf_catastro["unidad_05"] = gdf_catastro["unidad_05"].astype(str)
gdf_catastro["variedad"] = gdf_catastro["variedad"].astype(str)
gdf_catastro["soca"] = gdf_catastro["soca"].astype(str)
gdf_catastro["textura"] = gdf_catastro["textura"].astype(str)
gdf_catastro["cultivo"] = gdf_catastro["cultivo"].astype(str)
gdf_catastro["area"] = gdf_catastro["area"].astype(float)

gdf_catastro_sync["idd"] = gdf_catastro_sync["idd"].astype(str)
gdf_catastro_sync["unidad_01"] = gdf_catastro_sync["unidad_01"].astype(str)
gdf_catastro_sync["unidad_02"] = gdf_catastro_sync["unidad_02"].astype(str)
gdf_catastro_sync["unidad_03"] = gdf_catastro_sync["unidad_03"].astype(str)
gdf_catastro_sync["unidad_04"] = gdf_catastro_sync["unidad_04"].astype(str)
gdf_catastro_sync["unidad_05"] = gdf_catastro_sync["unidad_05"].astype(str)
gdf_catastro_sync["variedad"] = gdf_catastro_sync["variedad"].astype(str)
gdf_catastro_sync["soca"] = gdf_catastro_sync["soca"].astype(str)
gdf_catastro_sync["textura"] = gdf_catastro_sync["textura"].astype(str)
gdf_catastro_sync["cultivo"] = gdf_catastro_sync["cultivo"].astype(str)
gdf_catastro_sync["area"] = gdf_catastro_sync["area"].astype(float)

In [198]:
datasets

{'canha_quemada': {'project': 65454, 'dataset': 51666},
 'plan_renovacion': {'project': 36292, 'dataset': 382781}}

In [199]:
# identificar registros nuevos
reg_nuevos = gdf_catastro[~gdf_catastro['idd'].isin(gdf_catastro_sync['idd'])]

# completa campos para registro
data_reg_nuevos = reg_nuevos[['idd']].copy()
data_reg_nuevos['operacion'] = 'C'
data_reg_nuevos['procesado'] = 'false'

# iterar en los datasets
all_reg_nuevos = None
for key in datasets:
    data_reg_nuevos['dataset'] = datasets[key]['dataset']
    data_reg_nuevos['project'] = datasets[key]['project']
    all_reg_nuevos = pd.concat([all_reg_nuevos, data_reg_nuevos], ignore_index=True)

# registro en base de datos
all_reg_nuevos.to_sql("registro_sync", obtener_engine(), schema="amigcloud_sync", if_exists="append", index=False)

0

In [200]:
# identificar registros eliminados
reg_eliminados = gdf_catastro_sync[~gdf_catastro_sync['idd'].isin(gdf_catastro['idd'])]

# completa campos para registro
data_reg_eliminados = reg_eliminados[['idd']].copy()
data_reg_eliminados['operacion'] = 'D'
data_reg_eliminados['procesado'] = 'false'

# iterar en los datasets
all_reg_eliminados = None
for key in datasets:
    data_reg_eliminados['dataset'] = datasets[key]['dataset']
    data_reg_eliminados['project'] = datasets[key]['project']
    all_reg_eliminados = pd.concat([all_reg_eliminados, data_reg_eliminados], ignore_index=True)

# registro en base de datos
all_reg_eliminados.to_sql("registro_sync", obtener_engine(), schema="amigcloud_sync", if_exists="append", index=False)

0

In [207]:
reg_comparacion = gdf_catastro_sync.merge(gdf_catastro, on='idd', how='inner', suffixes=('_cat', '_cat_sync'))
reg_modificados = reg_comparacion[
    (reg_comparacion['unidad_01_cat'] != reg_comparacion['unidad_01_cat_sync']) |
    (reg_comparacion['unidad_02_cat'] != reg_comparacion['unidad_02_cat_sync']) |
    (reg_comparacion['unidad_03_cat'] != reg_comparacion['unidad_03_cat_sync']) |
    (reg_comparacion['unidad_04_cat'] != reg_comparacion['unidad_04_cat_sync']) |
    (reg_comparacion['unidad_05_cat'] != reg_comparacion['unidad_05_cat_sync']) |
    (reg_comparacion['variedad_cat']  != reg_comparacion['variedad_cat_sync'])  |
    (reg_comparacion['soca_cat']      != reg_comparacion['soca_cat_sync'])      |
    (reg_comparacion['textura_cat']   != reg_comparacion['textura_cat_sync'])   |
    (reg_comparacion['cultivo_cat']   != reg_comparacion['cultivo_cat_sync'])   |
    (np.round(reg_comparacion['area_cat'], 0) != np.round(reg_comparacion['area_cat_sync'], 0))
]

len(reg_modificados)

0

In [208]:
reg_modificados

,idd,geom_cat,unidad_01_cat,unidad_02_cat,unidad_03_cat,unidad_04_cat,unidad_05_cat,variedad_cat,soca_cat,textura_cat,...,unidad_01_cat_sync,unidad_02_cat_sync,unidad_03_cat_sync,unidad_04_cat_sync,unidad_05_cat_sync,variedad_cat_sync,soca_cat_sync,textura_cat_sync,cultivo_cat_sync,area_cat_sync


In [209]:

# completa campos para registro
data_reg_modificados = reg_modificados[['idd']].copy()
data_reg_modificados['operacion'] = 'U'
data_reg_modificados['procesado'] = 'false'

# iterar en los datasets
all_reg_modificados = None
for key in datasets:
    data_reg_modificados['dataset'] = datasets[key]['dataset']
    data_reg_modificados['project'] = datasets[key]['project']
    all_reg_modificados = pd.concat([all_reg_modificados, data_reg_modificados], ignore_index=True)

# registro en base de datos
all_reg_modificados.to_sql("registro_sync", obtener_engine(), schema="amigcloud_sync", if_exists="append", index=False)

0

In [127]:
lote = get_registro_sync()
lote

,id,idd,operacion,dataset,procesado,project
0,12001,14966,C,51666,False,65454


In [128]:
if lote['operacion'][0] == 'C':
    None
elif lote['operacion'][0] == 'D':
    None
elif lote['operacion'][0] == 'U':
    None
else:
    print('Operación no definida')

In [101]:
#create

In [129]:
lote_new = get_lote_by_idd(lote['idd'][0])
lote_new

,idd,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,variedad,soca,textura,cultivo,area
0,14966,"MULTIPOLYGON (((481092.293 8089434.165, 481685...",256.0,LOS ANDES--AGUILERA,794.0,AGUILERA WENDE MARIANO,LA18.2,UCG9020,2.0,FA,canha,14.676853


In [130]:
# repreyectar a WGS84
lote_new = lote_new.to_crs(epsg=4326)

In [131]:
lote_new

,idd,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,variedad,soca,textura,cultivo,area
0,14966,"MULTIPOLYGON (((-63.17790 -17.28024, -63.17232...",256.0,LOS ANDES--AGUILERA,794.0,AGUILERA WENDE MARIANO,LA18.2,UCG9020,2.0,FA,canha,14.676853


In [136]:
# convertir poligonos a multipoligonos
lote_new['geom'] = lote_new['geom'].apply(convertir_a_multipolygon)

In [137]:
lote_new

,idd,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,variedad,soca,textura,cultivo,area
0,14966,"MULTIPOLYGON (((-63.17790 -17.28024, -63.17232...",256.0,LOS ANDES--AGUILERA,794.0,AGUILERA WENDE MARIANO,LA18.2,UCG9020,2.0,FA,canha,14.676853


In [142]:
wkb_hex = convertir_a_wkb(lote_new['geom'][0])

In [143]:
insert_sql = f"""
    INSERT INTO dataset_345601 (id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, etiqueta, producto, dosis, cantidad, geometry)
    VALUES ({row['id']}, {row['unidad_01']}, '{row['unidad_02']}', {row['unidad_03']}, '{row['unidad_04']}', '{row['unidad_05']}', {row['area']}, '{row['origen']}', '{row['dias']}', '{row['os']}', '{str(round(row['area'],2)) + 'ha - ' + str(round(row['producto_cantidad'],2)) + 'ml'}', '{row['producto_plan']}', {row['producto_dosis']}, {row['producto_cantidad']}, ST_SetSRID(ST_GeomFromWKB('\\x{wkb_hex}'), 4326));
    """

'0106000000010000000103000000010000000D000000C3C57D5DC5964FC01485E3DEBD4731C02492DD730E964FC08D253BFDAF4731C0089603DA15964FC095D9DF33CC4731C0A61BB62019964FC01C1C38D0E24731C0B842E7661A964FC06DCC749BEB4731C093FE8C8A28964FC0C5DDA6C5314831C047D63E132B964FC02D93F4823A4831C00D48F608B6964FC044D1283C424831C029B53100BA964FC0A995B266424831C061E4B643DD964FC04AFB181D444831C048061BA7EA964FC0E0F209B1C04731C00D47BE5CC6964FC006B737F3BD4731C0C3C57D5DC5964FC01485E3DEBD4731C0'

In [ ]:
def ejecutar_query_sql(id_project, query):
    url_proyecto_sql = f'https://app.amigocloud.com/api/v1/projects/{id_project}/sql'
    query_sql = {'query': query}
    try:
        return amigocloud.post(url_proyecto_sql, query_sql)
    except Exception as e:
        print(f"Error al ejecutar la consulta SQL: {e}")
    return None

In [ ]:
B